# Biophysics Project

In [1]:
from Bio.PDB import *
from sys import stdin

# PLEASE INSTALL tqdm through pip, it will show progress bars of execution!
# Otherwise simply remove the tqdm from any iterator
from tqdm import tqdm

In [2]:
wd = input( "Please enter your working directory.\nIt should contain the cleaned PDB file. Remember the final '/' : " )

Please enter your working directory.
It should contain the cleaned PDB file. Remember the final '/' : /home/dbj/aa_school/biophysics_project/


Make sure this jupyter notebook exists in the same directory as the _cleaned-6m0j.pdb_ file!

In [3]:
st = PDBParser(PERMISSIVE=1).get_structure("6m0j", wd + "/6m0j_fixed.pdb")

# Interface as calculated by distance

#### The following functions relate to the generation of an interface of residues based on distance. The governing principal behind these functions is that given some threshold, we must identify residues at or below the specified distance between any given chains within our structure.

We will consider the distance between two residues as the smallest distance between any pair of atoms from the pair of aminoacids. 

Alternative ways of defining distance could be, for example, the distance between the center of mass of a given residue pair, which would be computationally cheaper.

    _____________________________________________________________

#### minimal_residue_distance ( a_amino, b_amino ) 
    This function receives as input two residues within the 
    structure, and returns the distance between them.

In [4]:
def minimal_residue_distance(a_amino, b_amino):
    smallest = None
    
    for a_atom in a_amino:
        for b_atom in b_amino:
            if smallest is not None:
                dist = a_atom - b_atom
                if dist < smallest:
                    smallest = dist
            else:
                smallest = a_atom - b_atom
    
    return(smallest)

    _____________________________________________________________
#### distances_chains( st )
    This function receives as input our cleaned PDB structure and 
    returns a dictionary of distances associated to every residue 
    pair. This step is computationally intensive but allows us to
    change the interface distance afterwards by accessing the
    generated dictionary.

In [5]:
def distances_chains(st):
    d_pairs = {}
    prev_chain = None

    for chain in st:

        if prev_chain is None:
            prev_chain = chain

        else:
            for a_residue in tqdm(st[prev_chain.id]):
                for b_residue in st[chain.id]:
                    dist = minimal_residue_distance(a_residue, b_residue)

                    if dist in d_pairs:
                        d_pairs[dist].append((a_residue, b_residue))
                    else:
                        d_pairs[dist] = [(a_residue, b_residue)]
    return(d_pairs)

    _____________________________________________________________
#### d_pairs
    This object will store the distance of all res-res pairs 
    between chains

In [6]:
d_pairs = distances_chains(st[0])

100%|██████████| 597/597 [01:10<00:00,  8.47it/s]


The benefit of using a jupyter notebooks is that the computational intensity of the function call *distances_chains * need only be executed  once during a session. After that, the genereated dictionary of pairs is stored in memory.

    _____________________________________________________________
#### interface(  threshold )
    This function will receive as input an integer represnting
    the threshold distance for our interface and return a pair
    of sets containing all residues in hypothetical chain A and
    hypothetical chain B. For the purposes of this project,
    chain "A" and chian "E" respectively.
    
    This function will take advantage of the pre-computed distance
    values

In [7]:
def interface(threshold):
    interface_residues = {"Total" : set(), "chain_A" : set(), "chain_B" : set()}

    for distance in sorted(d_pairs):
        if distance <= threshold:
            for pairs in d_pairs[distance]:
                interface_residues["Total"].add(pairs[0])
                interface_residues["Total"].add(pairs[1])
                interface_residues["chain_A"].add(pairs[0])
                interface_residues["chain_B"].add(pairs[1])
        else:
            return(interface_residues)

    return(interface_residues)

# Energy Calculations:

#### The following are functions and classess related to the calculation of energy. To do so, we will associate each atom with its known physico-chemical properties that are relevant for our energy calculations (such as charge, energy of solvation of external residues, Van derWaal parameters). 

    ____________________________________________________________

## Setup 
#### Associating atoms within our structure to known/calculated properties

In [8]:
import argparse
import sys
import os
import math

from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.NACCESS import NACCESS_atomic
from Bio.PDB.PDBIO import PDBIO, Select

In [9]:
class ResiduesDataLib():
    def __init__(self, fname):
        self.residue_data = {}
        try:
            fh = open(fname, "r")
        except OSError:
            print("#ERROR while loading library file (", fname, ")")
            sys.exit(2)
        for line in fh:
            if line[0] == '#':
                continue
            data = line.split()
            r = Residue(data)
            self.residue_data[r.id] = r
        self.nres = len(self.residue_data)

    def get_params(self, resid, atid):
        atom_id = resid + ':' + atid
        if atom_id in self.residue_data:
            return self.residue_data[atom_id]
        else:
            print("WARNING: atom not found in library (", atom_id, ')')
            return None

class Residue():
    def __init__(self,data):
        self.id     = data[0]+':'+data[1]
        self.at_type = data[2]
        self.charge  = float(data[3])
        
class AtomType():
    def __init__(self, data):
        self.id   = data[0]
        self.eps  = float(data[1])
        self.sig  = float(data[2])
        self.mass = float(data[3])
        self.fsrf = float(data[4])
        self.rvdw = self.sig * 0.5612
        
class VdwParamset(): #extracted from GELPI's github
    #parameters for the VdW
    def __init__ (self, file_name):
        self.at_types = {}
        try:
            fh = open(file_name, "r")
        except OSError:
            print ("#ERROR while loading parameter file (", file_name, ")")
            sys.exit(2)
        for line in fh:
            if line[0] == '#':
                continue
            data = line.split()
            self.at_types[data[0]] = AtomType(data)
        self.ntypes = len(self.at_types)
        fh.close()

### Notice the file extensions
File extensions are irrelevant as it pertains to amino-acid library and vanderwaal's parameter files. For the script to work, either modify the file names to reflect your currentfollowing two blocks of code OR ensure that the files aaLib and vdwprm are in txt format

In [10]:
# loading residue library from data/aaLib.lib
residue_library = ResiduesDataLib( wd + 'aaLib.txt')

In [11]:
# loading VdW parameters
ff_params = VdwParamset( wd + 'vdwprm.txt')

In [12]:
def residue_id(res):
    '''Returns readable residue id'''
    return '{} {}{}'.format(res.get_resname(), res.get_parent().id, res.id[1])

def atom_id(at):
    '''Returns readable atom id'''
    return '{}.{}'.format(residue_id(at.get_parent()), at.id)

def add_atom_parameters(st, res_lib, ff_params):
    ''' Adds parameters from libraries to atom .xtra field
        For not recognized atoms, issues a warning and put default parameters
    '''
    for at in st.get_atoms():
        resname = at.get_parent().get_resname()
        params = res_lib.get_params(resname, at.id)
        if not params:
            print("WARNING: residue/atom pair not in library ("+atom_id(at) + ')')
            at.xtra['atom_type'] = at.element
            at.xtra['charge'] = 0
        else:
            at.xtra['atom_type'] = params.at_type
            at.xtra['charge'] = params.charge
        at.xtra['vdw'] = ff_params.at_types[at.xtra['atom_type']]

### Add atom parameters Warnings
    If the execution of the following block of code produces some type of warning of the type 
    "WARNING:atom not found in library ... " this means that either: 
        
        there are atoms in the parsed PDB file that do not exist in the amino acid library file, 
        OR 
        the atoms in the aaLib file have different names than those of the parsed structure.
        
    If the latter is the case, manually edit the aaLib file to reflect the atoms of your parsed PDB file.

In [13]:
add_atom_parameters(st, residue_library, ff_params)

# Important : NACCESS

If NACCESS does not exist in your working directory, follow from step 3.

Otherwise if some type of error emerges when executing the block of code below, the easiest way to trouble shoot would be to reinstall NACCESS from the beginning and recompute the relevant surface:

    1) Make sure you're in the correct working directory ( should contain the PDB file )
    
    2) Delete the malfunctioning NACCESS ( you may need to do "sudo rm -r soft" )
        rm -r soft
        
    3) Download the NACCESS files and extract the "soft" directory into your working directory
    
    4) Install the naccess utility through csh ( you may need to download it if it isn't present in your system )
        csh install.scr
    
    5) Compute the relevant surface to genereate an ASA file that indicates the
       accessibility of atoms within our structure:
        ./naccess ../../6m0j_fixed.pdb

    6) Check that the .asa file has been created!
    

In [14]:
parser = PDBParser(PERMISSIVE=1)

NACCESS_BINARY = wd + 'soft/NACCESS/naccess'

srf = NACCESS_atomic(st[0], naccess_binary=NACCESS_BINARY)

# Prepare surfaces for the separate chains
# Alternatively the twp PDB files can be prepared outside and parsed here

io = PDBIO()
st_chains = {}
# Using BioIO trick (see tutorial) to select chains
class SelectChain(Select):
    def __init__(self, chid):
        self.id = chid

    def accept_chain(self, chain):
        if chain.id == self.id:
            return 1
        else:
            return 0

for ch in st[0]:
    io.set_structure(st)
    io.save('tmp.pdb', SelectChain(ch.id))
    st_chains[ch.id] = parser.get_structure('stA', 'tmp.pdb')
    add_atom_parameters(st_chains[ch.id], residue_library, ff_params)
    srfA = NACCESS_atomic(st_chains[ch.id][0], naccess_binary=NACCESS_BINARY)
os.remove('tmp.pdb')

# Computing Energies:

    For the purposes of reusability, the following functions 
    have variables with names such as resA resB or atom_a atom_b. 

    These names simply indicate that the given residue/atom 
    pertains to a different chain - not to be confused with 
    the actual chain names in our PDB structures (in our 
    working example, chain A and chain E).


#### For the sake of accessibility later on in the workflow, any function that calculates energy associated with a given residue will check whether the current atom is present in the ala_atoms set. If so, we can consider a seperate total energy only considering these atoms as the energy contribution that mutating the given residue to Alanine would produce.

In [15]:
ala_atoms = {'N', 'H', 'CA', 'HA', 'C', 'O', 'CB', 'HB', 'HB1', 'HB2', 'HB3', 'HA1', 'HA2', 'HA3'}

### Electrostatics and Van derwaals components:

In [16]:
def MH_diel(r):
    '''Mehler-Solmajer dielectric'''
    return 86.9525 / (1 - 7.7839 * math.exp(-0.3153 * r)) - 8.5525

def elec_int(at1, at2, r):
    '''Electrostatic interaction energy between two atoms at r distance'''
    return 332.16 * at1.xtra['charge'] * at2.xtra['charge'] / MH_diel(r) / r

def vdw_int(at1, at2, r):
    '''Vdw interaction energy between two atoms'''
    eps12 = math.sqrt(at1.xtra['vdw'].eps * at2.xtra['vdw'].eps)
    sig12_2 = at1.xtra['vdw'].sig * at2.xtra['vdw'].sig
    return 4 * eps12 * (sig12_2**6/r**12 - sig12_2**3/r**6)

### residue_interaction_energy ( a_amino, b_amino)
Recieves an amino acid pair as input to calculate associated interaction energy, returns the vanderwaals and electrostatic interactions between these residue pairs and the energy of interaction when a_amino is "mutated" for alanine.


In [17]:
def residue_interaction_energy(a_amino, b_amino):
    energy     = {'elec' : 0, 'vdw' : 0, 'total' : 0}
    ala_energy = {'elec' : 0, 'vdw' : 0, 'total' : 0}

    for a_atom in a_amino:
        for b_atom in b_amino:
            r    = a_atom - b_atom
            elec = elec_int(a_atom, b_atom, r)
            vdw  = vdw_int( a_atom, b_atom, r)
            
            energy['elec' ] += elec
            energy['vdw'  ] += vdw
            energy['total'] += vdw + elec

            
            if a_atom.id in ala_atoms:
                ala_energy['elec' ] += elec
                ala_energy['vdw'  ] += vdw
                ala_energy['total'] += vdw + elec

    return(energy, ala_energy)

### calc_solvation (st, res)
Receives a structure and residue as input. 
Returns a tuple with the energy of solvation of said residue with respect to the structure for the unmutated and ala-mutated case
    
    Important: when calculating the solvation of the seperate
    structures (i.e solvation of chains NOT of complex) we utilize
    a previously created object st_chains that only contains the 
    residues of the specified chain

In [18]:
def calc_solvation(st, res):
    '''Solvation energy based on ASA'''
    solv = 0
    solv_ala = 0
    for at in res.get_atoms():
        if 'EXP_NACCESS' not in at.xtra:
            continue
        s = float(at.xtra['EXP_NACCESS'])* at.xtra['vdw'].fsrf
        solv += s
        if at.id in ala_atoms:
            solv_ala += s
    return solv, solv_ala


## Total Interaction Energy (considering all residues):

Below we have have a function that will iterate across all residues in chainA, and for each residue in chainA it will iterate across each residue in chainB, adding the corresponding interaction and slolvation energies.

Note that the way we've factored the function is for legibility and clarity of the underlying computations. It would clearly be more efficient to calculate ABsolv and the Asolv/Bsolv in the outer for loop.
Additionally

In [241]:
def calc_total_energy():
    total_energy = 0

    for resA in tqdm(st[0]["A"]):
        for resB in st[0]["E"]:
            ch = resA.get_parent()
        
            elec_vdw = residue_interaction_energy( resA  , resB )[0] # take position [0] because that corresponds to unmutated energy of resA
            ABsolv   = calc_solvation( st[0]             , resA )[0] #  ''     ''     ''    ''    ''      ''      ''    ''       ''   ''  ''
            Asolv    = calc_solvation( st_chains[ ch.id ], resA )[0]

            total_energy += elec_vdw["total"] + ABsolv - Asolv

    for resB in tqdm(st[0]["E"]):
        for resA in st[0]["A"]:
            ch = resB.get_parent()
        
            elec_vdw = residue_interaction_energy( resB   , resA )[0] # take position [0] because that corresponds to unmutated energy of resB
            ABsolv   = calc_solvation( st[0]              , resB )[0] #  ''     ''     ''    ''    ''      ''      ''    ''       ''   ''  ''
            Bsolv    = calc_solvation( st_chains[ ch.id ] , resB )[0]
        

            total_energy += elec_vdw["total"] + ABsolv - Bsolv

    
    print ( f' Total energy of interaction considering all residues is {total_energy}' )
    return( total_energy )

In [242]:
calc_total_boolean = input("Do you want to calculate all energy considering all residues?\nThe computatinon will take ~ 5 minutes\nAnswer Y/N\n\n")

if calc_total_boolean == "Y":
    total_energy = calc_total_energy()
else:
    pass

Do you want to calculate all energy considering all residues?
The computatinon will take ~ 5 minutes
Answer Y/N

Y


100%|██████████| 194/194 [05:27<00:00,  1.69s/it]

 Total energy of interaction considering all residues is -180.7173174234688


## Interface Interaction Energy (considering interface residues):

#### Identify Interface residues at or below threshold distance:

In [19]:
threshold_dist = float(input("Specify threshold distance for interface :\n"))

Specify threshold distance for interface :
8


In [20]:
interface_residues = interface(threshold_dist)

###  Calculate Interface Energy

Below we show Jelpi's structure for the code block underneath this text

    for ch in structure:
        for res in ch.get_residues():

            if threshold > 0 and res not in interface[ch]:
                continue
            
            /*CALCULATE ENERGIES*/
         
For the sake of clarity of our underlying calculations, we decided to directly do two for loops instead of implemnting _continue_ to proceed with iterating through all residues of the interface. We believe the underlying concept is more clearly seen where we iterate in the way that is done below.

In [31]:
def interface_energy_simple():

    int_energy = 0
    
    for resA in tqdm(interface_residues["chain_A"]):
        for resB in interface_residues["chain_B"]:
        
            ch = resA.get_parent()    # let ch be the chain we'll consider
        
            elec_vdw = calc_int_energies( st[0] , resB ) # electric and vanderwaals interaction between resA and resB
            ABsolv   = calc_solvation(       st[0]      , resA ) # energetic contribution to solvation of resA in proteinA-proteinB complex
            Asolv    = calc_solvation( st_chains[ch.id] , resA ) # energetic contribution to solvation of resA in proteinA
        
            # add to total energy the energetic contribution of resA in interface interaction
            int_energy += elec_vdw[0] + elec_vdw[0]  + ABsolv[0] - Asolv[0]  

    for resB in tqdm(interface_residues["chain_B"]):
        for resA in interface_residues["chain_A"]:
            
            ch = resB.get_parent()   # let ch be the chain we'll consider
        
            elec_vdw = calc_int_energies( st[0] , resA ) # electric and vanderwaals interaction between resB and resA
            ABsolv   = calc_solvation(       st[0]      , resB ) # energetic contribution to solvation of resB in proteinA-proteinB complex
            Bsolv    = calc_solvation( st_chains[ch.id][0][ch.id][resB.id[1]] , resB ) # energetic contribution to solvation of resB in proteinB

            # add to total energy the energetic contribution of resB in interface interaction
            int_energy += elec_vdw[0] + elec_vdw[2] + ABsolv[0] - Bsolv[0]

    return(int_energy)

In [33]:
print(f"Interaction energy considering only interface residues at a threshold distance of {threshold_dist} Angstrom is {interface_energy_simple()}")

  0%|          | 0/61 [01:07<?, ?it/s]


KeyboardInterrupt: 

In [27]:
def calc_int_energies(st, res):
    '''Returns interaction energies (residue against other chains)
        for all atoms and for Ala atoms
    '''
    elec = 0.
    elec_ala = 0.
    vdw = 0.
    vdw_ala = 0.

    for at1 in res.get_atoms():
        for at2 in st.get_atoms():
            # skip same chain atom pairs
            if at2.get_parent().get_parent() != res.get_parent():
                r = at1 - at2
                e = elec_int(at1, at2, r)
                elec += e
                if at1.id in ala_atoms: #GLY are included implicitly
                    elec_ala += e
                e = vdw_int(at1, at2, r)
                vdw += e
                if at1.id in ala_atoms: #GLY are included implicitly
                    vdw_ala += e
    return elec, elec_ala, vdw, vdw_ala

In [24]:
Ala = dict( )

for res in tqdm(interface_residues["Total"]):
    ch = res.get_parent()
    
    int_energy  = calc_int_energies(st[0], res)
    ABsolv      = calc_solvation( st[0], res)
    CHsolv      = calc_solvation( st_chains[ch.id][0][ch.id][res.id[1]] , res )
    
    DG = int_energy[1] - int_energy[0] + int_energy[3] - int_energy[2] + ABsolv[1] - ABsolv[0] + CHsolv[1] - CHsolv[0]
    
    # this step of checking whether we've computed the same DG before is (more than anything) to prevent the loss of Alanines or Glycines that would all have a delta G of 0
    if DG in Ala:
        Ala[DG].append(f'{res.get_parent().id}, {res.resname}, {res.id[1]}, {DG}, {int_energy[1]:10f}, {int_energy[0]:10f}, {int_energy[3]:10f}, {int_energy[2]:10f}, {ABsolv[1]:10f}, {ABsolv[0]:10f}, {CHsolv[1]:10f}, {CHsolv[0]:10f}, {res}')
    else:
        Ala[DG] = [f'{res.get_parent().id}, {res.resname}, {res.id[1]}, {DG}, {int_energy[1]:10f}, {int_energy[0]:10f}, {int_energy[3]:10f}, {int_energy[2]:10f}, {ABsolv[1]:10f}, {ABsolv[0]:10f}, {CHsolv[1]:10f}, {CHsolv[0]:10f}, {res}']

100%|██████████| 112/112 [02:18<00:00,  1.23s/it]


In [488]:
tsv = input("Would you like to create a TSV file containing the ΔG of each residue in the interface you defined previously?\n")

Would you like to create a TSV file containing the ΔG of each residue in the interface you defined previously?
Y


In [490]:
if tsv == "Y":
    file_name = f"{wd}/MutationalAnalysis_IntefaceOfDistance_{threshold_dist}.tsv"
    with open(file_name, "w") as f:
        f.write('Chain, Residue, ResID,  ΔG_of_ALA_mutation, ALA_Electrostatics, Non-ALA_Electrostatics, ALA_VdW, Non-ALA_Vdw, ALA_AB Solvation, Non-ALA_AB Solvation , ALA_Chain_Solvation, Non-ALA_Chain_Solvation, Residue_Identifier\n')
        for DG in sorted(Ala):
            for line in Ala[DG]:
                f.write(line)
                f.write('\n')
        file.close()

In [260]:
for DG in sorted(Ala):
    print(Ala[DG]+'\n', end ="")


E, TYR, 449, -11.229097268216996,  -0.050953,   0.818172,  -0.149854,  -0.788425,  -0.171422,   5.327850,  -0.171422,   5.327850, <Residue TYR het=  resseq=449 icode= >
E, TYR, 453, -6.010665970015246,  -0.026278,   0.034926,  -0.120213,   4.887734,   0.000000,   0.470757,   0.000000,   0.470757, <Residue TYR het=  resseq=453 icode= >
E, VAL, 445, -3.7736334582931352,  -0.622038,  -0.053178,  -0.350741,  -0.693951,  -0.635547,   1.138445,  -0.635547,   1.138445, <Residue VAL het=  resseq=445 icode= >
E, PHE, 486, -3.173314339644346,  -0.038136,   0.024602,  -1.480352,  -7.117907,  -0.978365,   3.395701,  -0.978365,   3.395701, <Residue PHE het=  resseq=486 icode= >
A, MET, 82, -1.5096483705370924,   0.015375,   0.009605,  -0.236030,  -1.712731,  -0.146402,   1.349658,  -0.146402,   1.349658, <Residue MET het=  resseq=82 icode= >
A, LEU, 79, -0.3074086770259328,   0.019698,   0.028589,  -0.824890,  -1.840071,   0.000000,   0.656849,   0.000000,   0.656849, <Residue LEU het=  resseq=79 i

### Determine the energy contribution of sequentially further residues
Imagine we wanted to know (starting from the closest resA-resB pair) what the energy contribution of each residue is. This step is quite straightforward, instead of calculating the interface interaction energy once, we will do so for 

We will utilize the function *calc_int_energies()* as a template, and create a function that will work much the same, except this time we will be utilizing an interface that will be continuosly updated with the next residue pair with smallest distance

In [491]:
def calc_temp_energy( ):
    energy_contribution = 0
    
    for resA, resB, dist in tqdm(temp_interf["chainA"]):
            
        ch = resA.get_parent()
    
        int_energy  = calc_int_energies(st[0], resA)
        ABsolv      = calc_solvation( st[0], resA)
        CHsolv      = calc_solvation( st_chains[ch.id][0][ch.id][resA.id[1]] , resA )
    
        DG = int_energy[1] - int_energy[0] + int_energy[3] - int_energy[2] + ABsolv[1] - ABsolv[0] + CHsolv[1] - CHsolv[0]
        
        energy_contribution = int_energy[0] + int_energy[2] + ABsolv[0] - CHsolv[0] #This represents the energetic contribution to interaction energy of resA

        
        line =  f'{resA.get_parent().id}, {resA.resname}, {resA.id[1]}, {DG},'
        line += f' {int_energy[1]:10f}, {int_energy[0]:10f}, {int_energy[3]:10f},' 
        line += f' {int_energy[2]:10f}, {ABsolv[1]:10f}, {ABsolv[0]:10f}, {CHsolv[1]:10f}, {CHsolv[0]:10f},'
        line += f' {resA.get_parent().id}, {resA.id[1]}, {resA.resname}, {resB.get_parent().id}, {resB.id[1]}, {resB.resname}, {energy_contribution:.5e}, {dist}'
        
        temp_interf['holder'][resA] = line
    
    
    for resB, resA, dist in tqdm(temp_interf["chainB"]):
            
        ch = resB.get_parent()
        
        int_energy  = calc_int_energies(st[0], resB)
        ABsolv      = calc_solvation( st[0], resB)
        CHsolv      = calc_solvation( st_chains[ch.id][0][ch.id][resB.id[1]] , resB )
        
        DG = int_energy[1] - int_energy[0] + int_energy[3] - int_energy[2] + ABsolv[1] - ABsolv[0] + CHsolv[1] - CHsolv[0]
            
        energy_contribution = int_energy[0] + int_energy[2] + ABsolv[0] - CHsolv[0] #This represents the energetic contribution to interaction energy of resB

        line =  f'{resB.get_parent().id}, {resB.resname}, {resB.id[1]}, {DG},'
        line += f' {int_energy[1]:10f}, {int_energy[0]:10f}, {int_energy[3]:10f},' 
        line += f' {int_energy[2]:10f}, {ABsolv[1]:10f}, {ABsolv[0]:10f}, {CHsolv[1]:10f}, {CHsolv[0]:10f},'
        line += f' {resA.get_parent().id}, {resA.id[1]}, {resA.resname}, {resB.get_parent().id}, {resB.id[1]}, {resB.resname}, {energy_contribution:.5e}, {dist}'
        
        temp_interf['holder'][resB] = line
        


Below we see an object *temp_interf* which will serve as the repository for consequtively furhter residues. We will iterate across all resA-resB pairs, more concretely we will iterate across a sorted list of distances corresponding to resA-resB pairs.

If at any point one of the residues within the pair has not been seen before, we add both of them to their respective lists of _'chainA'_ and _'chainB'_, and most importantly we add them to the set of previously seen residues _'tot'_.

By the end of the following code block, we will have a list of residues in order of progressively further distances between the given residue and any pair of residues seen before.

In [492]:
temp_interf = {'tot' : set(), 'chainA' : [], 'chainB' : [], 'out' : [], 'holder' : dict()}


for dist in sorted(d_pairs):
    for resA, resB in d_pairs[dist]:
        A, B = False, False
        
        if resA not in temp_interf['tot']:
            A = True
            temp_interf['out'].append(resA)
            temp_interf['chainA'].append((resA, resB, dist))
            temp_interf['holder'][resA] = None
            
                
        if resB not in temp_interf['tot']:
            B = True
            temp_interf['out'].append(resB)
            temp_interf['chainB'].append((resB, resA, dist))
            temp_interf['holder'][resB] = None
            
        
        if A or B:
            temp_interf['tot'].add(resA)
            temp_interf['tot'].add(resB)

In [493]:
calc_temp_energy( )

100%|██████████| 194/194 [06:01<00:00,  1.86s/it]


In [494]:
first_line = 'Chain, Residue, ResID,  ΔG_of_ALA_mutation, '
first_line += 'ALA_Electrostatics, Non-ALA_Electrostatics, ALA_VdW, Non-ALA_Vdw, '
first_line += 'ALA_AB_Solvation, Non-ALA_AB_Solvation , ALA_Chain_Solvation, Non-ALA_Chain_Solvation, Residue_Identifier, '
first_line += 'ChainA, ResidueA, ResIDA, ChainB, ResidueB, Contributing_Interaction_Energy, Threshold_Distance\n'

file_name = f"{wd}/SequentialIntroductionThroughDistance.tsv"

with open(file_name, "w") as f:
    f.write(first_line)
    for res in temp_interf['out']:
        f.write(temp_interf['holder'][res] + '\n')


## Considering differnce in interaction energy of mutations purely inside of the interface

Imagine we would want to understand the energetic contribution of each residue-residue interaction between chains at the interface level. See that the function _calc_int_energy_  from Gelpí's code base calculates the energy of a specified residue agains the entire opposite chain. For this, we will use our own *residue_interaction_energy* function.


ALA and ALA_ref will store the energetic contribution 
of each considered residue on the interface. 
    
ALA will store the energetic contribution of mutated residueA.
We will look at all residueA-residueB pairs where residueA 
has been "mutated" to Alanine by computing the energetic
contribution when we discard non-Alanine atoms of the residue.
    
ALA_ref will be a reference table to check the unmutated energetic
contribution of residueA to the interaction energy.


In [228]:
ALA = dict()
ALA_ref = dict()
interf_energy = 0

The following code block will generate the necessary information to compute the desired

In [229]:
for resA in tqdm(interface_residues["chain_A"]):
    for resB in interface_residues["chain_B"]:
        
        ch = resA.get_parent()    # let ch be the chain we'll consider
        
        elec_vdw = residue_interaction_energy( resA , resB ) # electric and vanderwaals interaction between resA and resB
        ABsolv   = calc_solvation(       st[0]      , resA ) # energetic contribution to solvation of resA in proteinA-proteinB complex
        Asolv    = calc_solvation( st_chains[ch.id] , resA ) # energetic contribution to solvation of resA in proteinA
        
        if resA not in ALA:
            ALA[resA]     = dict()
            ALA_ref[resA] = dict()

        # record the interaction energy components of Alanine mutation on resA
        ALA[resA][resB]     = {   'elec' : elec_vdw[1]['elec'], 
                                   'vdw' : elec_vdw[1]['vdw' ],
                                'ABsolv' :   ABsolv[1], 
                                'CHsolv' :    Asolv[1], 
                                 'Total' : elec_vdw[1]['elec'] + elec_vdw[1]['vdw' ] + ABsolv[1] - Asolv[1] 
                                }
        
        # record the interaction energy components for non-mutated resA
        ALA_ref[resA][resB] = {   'elec' : elec_vdw[0]['elec'], 
                                   'vdw' : elec_vdw[0]['vdw' ],
                                'ABsolv' :   ABsolv[0], 
                                'CHsolv' :    Asolv[0], 
                                 'Total' : elec_vdw[0]['elec'] + elec_vdw[0]['vdw' ] + ABsolv[0] - Asolv[0] 
                                }

        # add to total energy the energetic contribution of resA in interface interaction
        interf_energy += elec_vdw[0]['elec'] + elec_vdw[0]['vdw']  + ABsolv[0] - Asolv[0]  

for resB in tqdm(interface_residues["chain_B"]):
    for resA in interface_residues["chain_A"]:
            
        ch = resB.get_parent()   # let ch be the chain we'll consider
        
        elec_vdw = residue_interaction_energy( resB , resA ) # electric and vanderwaals interaction between resB and resA
        ABsolv   = calc_solvation(       st[0]      , resB ) # energetic contribution to solvation of resB in proteinA-proteinB complex
        Bsolv    = calc_solvation( st_chains[ch.id][0][ch.id][resB.id[1]] , resB ) # energetic contribution to solvation of resB in proteinB

        if resB not in ALA:
            ALA[resB]     = dict()
            ALA_ref[resB] = dict()
        
        # record the interaction energy components of Alanine mutation on resB
        ALA[resB][resA]     = {   'elec' : elec_vdw[1]['elec'], 
                                   'vdw' : elec_vdw[1]['vdw' ],
                                'ABsolv' :   ABsolv[1], 
                                'CHsolv' :    Bsolv[1], 
                                 'Total' : elec_vdw[1]['elec'] + elec_vdw[1]['vdw' ] + ABsolv[1] - Bsolv[1] 
                                }
        # record the interaction energy components of non-mutated resB
        ALA_ref[resB][resA] = {   'elec' :   elec[0]['elec'], 
                                   'vdw' :    vdw[0]['vdw' ],
                                'ABsolv' : ABsolv[0], 
                                'CHsolv' :  Bsolv[0], 
                                 'Total' :   elec_vdw[0]['elec'] + elec_vdw[0]['vdw' ] + ABsolv[0] - Bsolv[0] 
                              }

        # add to total energy the energetic contribution of resB in interface interaction
        interf_energy += elec_vdw[0]['elec'] + elec_vdw[0]['vdw'] + ABsolv[0] - Bsolv[0]

100%|██████████| 51/51 [00:09<00:00,  5.30it/s]


#### Quantifying Energetic contribution of mutations of interface, on interface energy

The following two code blocks will

In [230]:
mutation_effect = dict()

for rA in ALA:
    mutation_energy = 0
    for rB in ALA[rA]:
        # calculate the energy associated with a mutation by accessing the precomputed values in ALA and ALA_ref
        residue = ALA_ref[rA][rB], ALA[rA][rB]
        mutation_energy += residue[1]["Total"] - residue[0]["Total"]
    
    if mutation_energy in mutation_effect:
        mutation_effect[mutation_energy].add( (rA, f"{rA.get_resname()} {rA.id[1]}" , rA.get_parent().id) )
    else:
        mutation_effect[mutation_energy] = {(rA, f"{rA.get_resname()} {rA.id[1]}" , rA.get_parent().id)}

In [231]:
for mutation_energy in sorted(mutation_effect):
    for mres in mutation_effect[mutation_energy]:
        print(mutation_energy, mres)

-5.059877861162056 (<Residue TYR het=  resseq=453 icode= >, 'TYR 453', 'E')
-2.072548908596613 (<Residue HIE het=  resseq=34 icode= >, 'HIE 34', 'A')
-0.5982479548902875 (<Residue GLU het=  resseq=484 icode= >, 'GLU 484', 'E')
-0.4604370171376484 (<Residue LYS het=  resseq=26 icode= >, 'LYS 26', 'A')
-0.4494323146889567 (<Residue GLU het=  resseq=406 icode= >, 'GLU 406', 'E')
-0.34211523649278197 (<Residue SER het=  resseq=19 icode= >, 'SER 19', 'A')
-0.2856942974799548 (<Residue ASP het=  resseq=405 icode= >, 'ASP 405', 'E')
-0.2701932350159772 (<Residue TYR het=  resseq=449 icode= >, 'TYR 449', 'E')
-0.13129095347856562 (<Residue SER het=  resseq=443 icode= >, 'SER 443', 'E')
-0.07712264202187213 (<Residue SER het=  resseq=494 icode= >, 'SER 494', 'E')
-0.06815914430902886 (<Residue PRO het=  resseq=389 icode= >, 'PRO 389', 'A')
-0.0644207676876149 (<Residue THR het=  resseq=478 icode= >, 'THR 478', 'E')
-0.02234616409392698 (<Residue PRO het=  resseq=84 icode= >, 'PRO 84', 'A')
-0.0